In [16]:
import pandas as pd
import json
import os
import numpy as np

In [2]:
os.chdir('/home/pi/dev/notebooks/30DayMap2024/data/kzoo_street_12225')
dirs = os.listdir()
os.chdir(dirs[0])
os.getcwd()

'/home/pi/dev/notebooks/30DayMap2024/data/kzoo_street_12225/1861928_West_Main_Street'

In [13]:
files = os.listdir()
df = pd.read_csv(files[0])
df.head(2)

,Data Periods,Mode of Travel,Zone ID,Zone Name,Road Classification,Line Zone Length (Miles),Zone Is Pass Through,Zone Direction (degrees),Zone Cardinal Direction,Zone is Bi-Direction,Day Type,Day Part,Average Daily Segment Traffic (StL Volume),Avg Segment Speed (mph),Avg Segment Travel Time (sec),Free Flow Speed (mph),Vehicle Miles of Travel (StL Volume),Travel Time Index,Congested Segment,85th Speed Percentile
0,"Jan 01, 2022 - Dec 31, 2023",All Vehicles - StL All Vehicles Volume,1111503841,West Main Street / 111568112 / 7,Primary,0.031,yes,92,EAST,yes,0: All Days (M-Su),00: All Day (12am-12am),11509,40,3,44.303,360.04,1.12,False,45
1,"Jan 01, 2022 - Dec 31, 2023",All Vehicles - StL All Vehicles Volume,1111503841,West Main Street / 111568112 / 7,Primary,0.031,yes,92,EAST,yes,0: All Days (M-Su),01: 12am (12am-1am),125,40,3,44.303,3.92,1.09,False,46


In [ ]:
df_all = df[(df['Day Part'] == '00: All Day (12am-12am)') & (df['Day Type'] == '0: All Days (M-Su)')]
c = df_all.columns
df_at = df_all[[c[2], c[10], c[12], c[13], c[15], c[19]]]
df_at.count()

,Zone ID,Day Type,Average Daily Segment Traffic (StL Volume),Avg Segment Speed (mph),Free Flow Speed (mph),85th Speed Percentile
0,1111503841,0: All Days (M-Su),11509,40,44.303,45
90,1111754274,0: All Days (M-Su),11450,37,42.516,43


In [146]:
url_path = 'https://raw.githubusercontent.com/eKerney/dataStore2/refs/heads/main/wayneGroundObs.geojson'
file_path = '/home/pi/dev/notebooks/30DayMap2024/data/kzoo_street_12225/1861928_West_Main_Street/Shapefile/1861928_West_Main_Street_osm_segment.geojson'
geojson = GeoJSONtools(file_path)
geojson.export_file("west_main")

Loaded JSON: 1861928_West_Main_Street_osm_segment.geojson
GeoJSON output: west_main.geojson


In [144]:
import pandas as pd
import geopandas as gpd
import os
import json
import numpy as np

class GeoJSONtools:
    def __init__(self, file_path: str=""):
        if not file_path:
            self.geojson = None
        elif file_path.startswith("https:") or file_path.startswith("http:"):
            gdf = gpd.read_file(file_path)
            self.geojson = json.loads(gdf.to_json())
        else:
            f = open(file_path)
            splt = file_path.split('/')
            print(f'Loaded JSON: {splt[-1]}')
            self.geojson = json.load(f)

    @property          
    def geojson(self): 
        return self._geojson

    @geojson.setter 
    def geojson(self, value):
        self._geojson = value  
    
    def export_file(self, file_name="geojson_file.geojson"):
        with open((f'{file_name}.geojson'), "w") as outfile:
            outfile.write(json.dumps(self.geojson))
        print(f'GeoJSON output: {file_name}.geojson')


In [44]:
geo_path = '/home/pi/dev/notebooks/30DayMap2024/data/kzoo_street_12225/1861928_West_Main_Street/Shapefile/1861928_West_Main_Street_osm_segment.geojson'
west_main_geo = load_json(geo_path)
west_main_geo['features'].__len__()

Loaded JSON: 1861928_West_Main_Street_osm_segment.geojson


53

In [106]:
df_at.set_index(df_at['Zone ID'], inplace=True)
data = df_at.to_dict('index')
data[1111503841]

{'Zone ID': 1111503841,
 'Day Type': '0: All Days (M-Su)',
 'Average Daily Segment Traffic (StL Volume)': 11509,
 'Avg Segment Speed (mph)': 40,
 'Free Flow Speed (mph)': 44.303,
 '85th Speed Percentile': 45}

In [ ]:
west_main_geo

In [ ]:
new_gj = {'type': 'FeatureCollection', 'features': []}
for f in west_main_geo['features']:
    f['properties'].update(data[f['properties']['segment_id']])
    new_gj['features'].append(f)
# new_gj

### geodata working section

In [1]:
import os
import importlib
os.chdir(os.path.expanduser('geodata-py-utils'))
# import geodata_py_utils
from geodata_py_utils import *
# importlib.reload(geodata_py_utils)

In [ ]:
GeoData.

### Testing input types

In [2]:
url_path = 'https://raw.githubusercontent.com/martynafford/natural-earth-geojson/refs/heads/master/110m/cultural/ne_110m_admin_0_countries.json'
geo_data = GeoData(url_path, InputTypes.DATA_URL, row_filter=2)
geo_data.gdf

,scalerank,featurecla,LABELRANK,SOVEREIGNT,SOV_A3,ADM0_DIF,LEVEL,TYPE,ADMIN,ADM0_A3,...,REGION_WB,NAME_LEN,LONG_LEN,ABBREV_LEN,TINY,HOMEPART,MIN_ZOOM,MIN_LABEL,MAX_LABEL,geometry
0,1,Admin-0 country,3.0,Afghanistan,AFG,0.0,2.0,Sovereign country,Afghanistan,AFG,...,South Asia,11.0,11.0,4.0,-99.0,1.0,0.0,3.0,7.0,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,1,Admin-0 country,3.0,Angola,AGO,0.0,2.0,Sovereign country,Angola,AGO,...,Sub-Saharan Africa,6.0,6.0,4.0,-99.0,1.0,0.0,3.0,7.0,"MULTIPOLYGON (((23.90415 -11.72228, 24.07991 -..."


In [3]:
url_path = 'https://raw.githubusercontent.com/eKerney/dataStore2/refs/heads/main/wayneGroundObs.geojson'
geo_data = GeoData(url_path, InputTypes.DATA_URL, row_filter=3)
geo_data.gdf.he

,FID,h3_index,DEM_elev,units,lidar_Z,Obs_AGL,Shape__Area,Shape__Length,geometry
0,1736,8a274d24b097fff,266.1,m,348.719,82.6,0.000002,0.004958,"POLYGON ((-83.54742 42.36547, -83.54667 42.365..."
1,2062,8a274d24c04ffff,267.3,m,320.970,53.7,0.000002,0.004962,"POLYGON ((-83.51675 42.40277, -83.516 42.40233..."
2,2063,8a274d24c04ffff,267.3,m,320.970,53.7,0.000002,0.004962,"POLYGON ((-83.51675 42.40277, -83.516 42.40233..."


In [2]:
file_path = '/home/pi/dev/notebooks/30DayMap2024/data/kzoo_street_12225/1861928_West_Main_Street/Shapefile/1861928_West_Main_Street_osm_segment.geojson'
geo_data = GeoData(file_path, InputTypes.DATA_PATH)
geo_data.gdf.head()

,segment_id,name,segment_ty,geometry
0,1111503841,West Main Street / 111568112 / 7,primary,"LINESTRING (-85.61777 42.29486, -85.61716 42.2..."
1,1111754274,West Main Street / 727695179 / 1,primary,"LINESTRING (-85.62555 42.2955, -85.62523 42.29..."
2,1111871475,West Main Street / 264373332 / 7,primary,"LINESTRING (-85.63055 42.29619, -85.62977 42.2..."
3,1112032394,West Main Street / 111568112 / 8,primary,"LINESTRING (-85.61716 42.29484, -85.61655 42.2..."
4,1112161013,West Main Street / 888230017 / 1,primary,"LINESTRING (-85.6616 42.29599, -85.66073 42.29..."


In [8]:
geo_data.export_geojson('../testing')

GeoJSON output: ../testing.geojson


In [12]:
geo_data.geojson['features'][0]

{'id': '0',
 'type': 'Feature',
 'properties': {'segment_id': 1111503841,
  'name': 'West Main Street / 111568112 / 7',
  'segment_ty': 'primary'},
 'geometry': {'type': 'LineString',
  'coordinates': [[-85.61777353286743, 42.29485583305359],
   [-85.61715984344482, 42.29484176635742]]}}